In [1]:
import h5py
import numpy as np

In [2]:
with h5py.File("runs/perfectionerer_more_agressa.hdf5", "r") as f:
    print(f.keys())
    print(f["properties"])
    print(f["p"])
    print(f["q"])
    print(f["x"][:])


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'runs/perfectionerer_more_agressa.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [3]:
def angular_band(pos, angle, width, xx, zz):
    return (pos-xx < angle*zz) * (pos + width - xx > angle*zz)

def angular_bands_on_drosophila(pos, width, xx,zz):
    angle = (pos/100.0 - 0.5)/3.
    return angular_band(pos, angle, width, xx, zz)

In [6]:
with h5py.File("runs/tenth_egg.hdf5", "r") as f:
    attrs = dict(f.attrs)
    poss = np.array(f["x"])
    cell_properties = np.array(f["properties"][0])
    p = np.array(f["p"])
    q = np.array(f["q"])




xx = poss[-1][:,0]
yy = poss[-1][:,1]
zz = poss[-1][:,2]

print(xx.min(), xx.max())
print(yy.min(), yy.max())
print(zz.min(), zz.max())

xx = (xx - xx.min())/(xx.max() - xx.min())*100
yy = (yy - yy.min())/(yy.max() - yy.min())*100
zz = (zz - zz.min())/(zz.max() - zz.min())*100

AB = lambda pos, width: angular_bands_on_drosophila(pos, width, xx, zz)

prop2 = np.zeros(cell_properties.shape)




width = 2.5
dist = 25

prop2[(xx > 30)*(xx < 101)*(zz < 40)] = 1.0 # Germ Band


for i in range(1,2):   #7
    xlow = 35 + i*dist
    w = width*(1+i)

    # prop2[(xx > xlow)*(xx < xhigh)*(zz > 30)] = 5.0 # 

    prop2[AB(xlow, w)*(zz>57)] = 5.0 # Dorsal Folds




prop2[AB(15, 9)*(zz<97)*(zz>45)] = 5.0 # Cephalic Furrow

# prop2[(xx > 30)*(xx < 97)*(zz < 50)*(yy< 50 + 40) * (yy > 50- 40)] = 1.0 # red


# prop2[(xx > 85)*(xx < 92)] = 4.0 # light blue tip


# prop2[angular_band(108, 0.3, 15, xx,zz)] = 3.0 # (PMG) Posterior Midgut Invagination

# prop2[angular_band(113, 0.3, 8, xx,zz)] = 4.0 # (PMG) Posterior Midgut Invagination

# prop2[angular_band(119, 0.3, 15, xx,zz)] = 3.0 # (PMG) Posterior Midgut Invagination
# prop2[(xx > 94)*(zz>63)*(yy<50+16)*(yy>50-16)] = 3.0 # yellow tip 


prop2[(yy < 65)*(yy > 35)*(xx > 15)*(xx < 90)*(zz < 44)] = 2.0 # Ventral Furrow

properties = prop2



with h5py.File("runs/stas_no_pmg.hdf5", "w") as f:
    create = lambda name, data: f.create_dataset(name, data=data)

    create("x", poss)

    create("properties", [properties])

    create("p", p)

    create("q", q)

    f.attrs.update(attrs)


-79.91615 59.94943
-26.649378 26.66619
-26.649714 26.656801


In [ ]:
# make 10_000 random points in a sphere

n = 10_000

r = np.random.uniform(0, 30, n)

phi, theta = np.random.uniform(0, 2*np.pi, n), np.arccos(2*np.random.uniform(0, 1, n) - 1)

x = r*np.sin(theta)*np.cos(phi)
y = r*np.sin(theta)*np.sin(phi)
z = r*np.cos(theta)

poss = np.array([x, y, z]).T

properties = np.zeros(n)

## p and q are n random unit vectors
p = np.random.normal(size=(n, 3)) 
p /= np.linalg.norm(p, axis=1)[:, None]

q = np.random.normal(size=(n, 3))
q /= np.linalg.norm(q, axis=1)[:, None]

with h5py.File("runs/random_sphere.hdf5", "w") as f:
    create = lambda name, data: f.create_dataset(name, data=data)

    create("x", poss)

    create("properties", [properties])

    create("p", p)

    create("q", q)


In [10]:
with h5py.File("runs/random_baby_3.hdf5", "r") as f:
    positions = f["x"][::2,:,:]

steps, N, _ = positions.shape
print(positions.shape)
# # flatten the positions
positions = positions.reshape(steps*N, 3)

# add the number og steps and N to the beginning of the array
properties = np.zeros((1, 3), dtype=float)

properties[0, 0] = N
properties[0, 1] = steps
properties[0, 2] = 0


savedata = np.concatenate([properties, positions], axis=0)


print(savedata.shape)
# # save the positions to a csv file
np.savetxt("runs/random_baby.csv", savedata, delimiter=",")

(101, 10000, 3)
(1010001, 3)
